<br></br>
# **나이브베이즈 모델의 활용**
1. **베이지안 평가 학습모델**을 불러온다 (0:부정 / 1:긍정)
1. **분석을 위한 네이버 뉴스 댓글**을 수집한다 (600개 내외가 적합)
1. **긍정/ 부정 댓글**을 분류한다
1. **긍정 리뷰**의 **Token** 을 트위터 **Stemming**으로 정규화 한다
1. **정규화된 Token** 중 **"명사", "동사", "형용사"** 만 수집
1. 수집된 Token List 객체를 **nltk.Text 로 변환 후 빈도상위 10개**를 출력한다

<img src="http://www.meconomynews.com/news/photo/201806/15255_14549_2116.png" />

<br></br>
## **1 모듈과 데이터 불러오기**
Import Data / Datum

In [1]:
# Step 1 : 앞에서 학습한 Classifier 모델을 불러온다
# >> classifier = pickle.

import pickle
file = './data/classifiers.model'
classifier = pickle.load(open(file, 'rb'))
classifier

In [2]:
# Step 2 : 분류기준 단어목록 불러오기
# >> selected_words = pickle.

file = './data/selected.words'
selected_words = pickle.load(open(file, 'rb'))
selected_words[:5]

['./Punctuation', '하다/Verb', '영화/Noun', '이/Josa', '보다/Verb']

In [3]:
# Step 3 : Tokenizer 생성/ 판단모듈
from konlpy.tag import Twitter
twitter = Twitter()

# 문단을 긍부정 분석용 Token 생성하기
def tokenize(doc):
    doc = ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]
    return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}

# 위에서 생성한 Token을 활용하여 판단하기
def classify_text(text):
    tokens = tokenize(text)             # 트위터로 Stemming/ Taggging 후 Selected_word 판단결과 객체
    return classifier.classify(tokens)  # 분류모델 평가

In [9]:
# Step 4 : 데이터와 모델, 함수를 활용하여 긍/부정 Test
# 0 : 부정모델 , 1 : 긍정모델

text   = """영화가 재미있어요"""
classify_text(text)

'1'

<br></br>
## **2 뉴스 데이터 댓글데이터 불러오기**
replies [참고](http://blog.naver.com/PostView.nhn?blogId=sangdo14&logNo=220610607049&categoryNo=0&parentCategoryNo=163&viewDate=&currentPage=1&postListTopCurrentPage=1&from=search)

In [11]:
from txtutil import Naver_news_rep
news_url = "https://news.naver.com/main/ranking/read.nhn?rankingType=popular_day&" +\
    "oid=001&aid=0010299155&date=20180827&type=1&rankingSectionId=101&rankingSeq=30"
    
data = Naver_news_rep(news_url)
print("Data length : {}\nData sample : {}".format(
    len(data), data[:5]))

Data length : 116
Data sample : ['"경제대국들의 무역전쟁이 이정도군. 한국정부는 돈이 너무 많아서 50조정도는 허공에 날리고 어디에 썼는지 데이타조차 없는데."', '"이제 좀 제대로 보네.. 뭐만 하면 11월 중간선거 때문에, 러시아스캔들때문에, 정치적으로 위기에 몰린 트럼프가 할수없이 자폭무역전쟁..별 희안한 기사만 써대더니..ㅉㅉ"', '"짱깨 뒷돈 받아 처먹은 참여련대, 청와대 주사파들, 그리고 한경오 쓰레기 언론들 어쩌냐ㅋㅋㅋㅋㅋ"', '"트럼프형 머찜. 찌질이 시진핑 더 밟아라."', '"문빨충 좌좀들아. 니들의 큰 아버지황제 시진핑이 위험하다. 어서 광화문에 모여서 좃불 좀비집회 해야지? 니들이 물고빠는 문재앙이 추앙하는 중국의 황제가 뚜까맞고 있자나! 어서 가서 좃불을 밝히세요"']


In [22]:
news_url

'https://news.naver.com/main/ranking/read.nhn?rankingType=popular_day&oid=001&aid=0010299155&date=20180827&type=1&rankingSectionId=101&rankingSeq=30'

<br></br>
## **Mission Test**
긍정 부정댓글 분류하기

In [6]:
# Mission 1
# 전체 댓글 갯수중 긍정/ 부정 갯수 계산하기

In [12]:
pro, con = [], []
for datum in data:
    if classify_text(datum) == "1":
        pro.append(datum)
    else:
        con.append(datum)

In [13]:
result = {}
result["긍정"] = pro
result["부정"] = con

In [14]:
len(result["긍정"])

46

In [15]:
len(result["부정"])

70

In [11]:
# Mission 2
# 긍정 댓글을 구분하여 분류한 뒤
# 명사, 형용사, 동사 Token만 추출하기 (전처리 / Token의 예외처리는 미포함)

In [12]:
%whos

Variable         Type                    Data/Info
--------------------------------------------------
Naver_news_rep   function                <function Naver_news_rep at 0x7fa986f55840>
Twitter          type                    <class 'konlpy.tag._twitter.Twitter'>
classifier       NaiveBayesClassifier    <nltk.classify.naivebayes<...>object at 0x7fa9bc1bd9b0>
classify_text    function                <function classify_text at 0x7fa986f559d8>
con              list                    n=80
data             list                    n=121
datum            str                     "아이고 중국이 그따위 모르고 전쟁 했는줄아냐<...>래라 개소리한다고 누깜짝 하는나라가 아니다."
file             str                     ./data/selected.words
news_url         str                     https://news.naver.com/ma<...>ctionId=101&rankingSeq=30
pickle           module                  <module 'pickle' from '/u<...>lib/python3.6/pickle.py'>
pro              list                    n=41
result           dict                    n=2
selected_words 

In [16]:
texts = result['긍정']
texts[:3]

['"트럼프형 머찜. 찌질이 시진핑 더 밟아라."',
 '"중국은  지금  확실하게  조지지  않으면  훗날  미국 최대의  위협국이  된다,,,,,!!!!!!!!!!!!!!!!!!"',
 '"중국은 하나의 중국이 아니라 10개의 중국으로 나눠져야한다  큰 땅에 여러민족이 있다 그중하나로 요동 지방은 고조선이라는 나라도 분리독립해야한다"']

In [17]:
print(twitter.pos(texts[2], stem=True))

[('"', 'Punctuation'), ('중국', 'Noun'), ('은', 'Josa'), ('하나', 'Noun'), ('의', 'Josa'), ('중국', 'Noun'), ('이', 'Josa'), ('아니다', 'Adjective'), ('10', 'Number'), ('개', 'Noun'), ('의', 'Josa'), ('중국', 'Noun'), ('으로', 'Josa'), ('나누다', 'Verb'), ('야하다', 'Adjective'), ('크다', 'Verb'), ('땅', 'Noun'), ('에', 'Josa'), ('여러', 'Noun'), ('민족', 'Noun'), ('이', 'Josa'), ('있다', 'Adjective'), ('그중', 'Adverb'), ('하나로', 'Noun'), ('요동', 'Noun'), ('지방', 'Noun'), ('은', 'Josa'), ('고조선', 'Noun'), ('이라는', 'Josa'), ('나라', 'Noun'), ('도', 'Josa'), ('분리', 'Noun'), ('독립', 'Noun'), ('하다', 'Verb'), ('하다', 'Verb'), ('"', 'Punctuation')]


In [18]:
for token in twitter.pos(texts[2], stem=True):
    if (token[1] == 'Noun') and (len(token[0]) >1):
        print(token[0])

중국
하나
중국
중국
여러
민족
하나로
요동
지방
고조선
나라
분리
독립


In [19]:
texts = result['긍정']
filtered_tokens = []
for text in texts:
    tokens = twitter.pos(text ,stem=True)
    tokens = [token[0]    for token in tokens    
                          if (token[1] in ['Noun','Adjective','Verb']) and (len(token[0]) > 1 )]
    filtered_tokens += tokens

In [20]:
# Mission 3
# nltk.Text() 객체를 활용하기
# 상위 빈도수 50인 Token을 추출하기

In [21]:
import nltk
pos_nltk = nltk.Text(filtered_tokens, name='긍정댓글')
pos_nltk.vocab().most_common(20)

[('하다', 32),
 ('트럼프', 16),
 ('중국', 15),
 ('미국', 14),
 ('되다', 10),
 ('한국', 7),
 ('있다', 6),
 ('전쟁', 6),
 ('시진핑', 5),
 ('짱깨', 5),
 ('조선시대', 5),
 ('민족', 4),
 ('잡다', 4),
 ('화이팅', 4),
 ('좋다', 4),
 ('들다', 4),
 ('무역', 4),
 ('우리', 4),
 ('지금', 3),
 ('아니다', 3)]

<br></br>
## **nltk 객체 활용하기**
Token 주변의 단어들 살펴보기

In [19]:
pos_nltk.concordance('조선시대')

Displaying 5 of 5 matches:
 역사 잊다 민족 미래 없다 하다 일제강점기 하다 중국 식민지 이다 조선시대 관심 없다 나라 조선시대 인조 배금 정책 보다 같다 중국 조공 바치
다 하다 일제강점기 하다 중국 식민지 이다 조선시대 관심 없다 나라 조선시대 인조 배금 정책 보다 같다 중국 조공 바치 국민 노예 삼다 풍악 즐
 배금 정책 보다 같다 중국 조공 바치 국민 노예 삼다 풍악 즐기다 조선시대 아마 기득권 세력 볼땐 조선시대 말로 살기 좋다 세상 아니다 싶다 
 바치 국민 노예 삼다 풍악 즐기다 조선시대 아마 기득권 세력 볼땐 조선시대 말로 살기 좋다 세상 아니다 싶다 역사 기억 하다 민족 생각 하다 
 살기 좋다 세상 아니다 싶다 역사 기억 하다 민족 생각 하다 미래 조선시대 같다 지금 전쟁 있다 무역 분쟁 보다 아침 시원하다 뉴스 트럼프 응
